In [1]:
!pip install pyarrow

     --------------------------------------- 20.6/20.6 MB 50.1 MB/s eta 0:00:00


In [1]:
#Fetch file from Promice folder (should be in same parent directory as GEUS-Master_Thesis)

import os
import pandas as pd

# Path to the directory containing CSV files
csv_directory = "..\..\PROMICE-AWS-toolbox\out\L4"

# List all CSV files in the directory
csv_files = [f for f in os.listdir(csv_directory) if f.endswith('.csv')]

# Combine all CSV files into a single DataFrame
dfs = []
for f in csv_files:
    df = pd.read_csv(os.path.join(csv_directory, f))
    df.insert(0, 'station_name', f[:-7])
    dfs.append(df)
df = pd.concat(dfs)

#read metadata from promice repository
station = pd.read_csv('..\..\PROMICE-AWS-toolbox\metadata\AWS_station_locations.csv')
station.to_csv('..\\data\\new_promice\\AWS_station_locations.csv', index=False)



output_file = "..\\Data\\new_promice\\all_promice_data.parquet.gzip"
df.to_parquet(output_file, compression='gzip', engine='pyarrow')

#read metadata from promice repository
station = pd.read_csv('..\..\PROMICE-AWS-toolbox\metadata\AWS_station_locations.csv')
station.to_csv('..\\data\\new_promice\\AWS_station_locations.csv', index=False)


#display(output station and columns summary)
display(df['station_name'].unique())
print(list(df.columns))


FileNotFoundError: [WinError 3] The system cannot find the path specified: '..\\..\\PROMICE-AWS-toolbox\\out\\L4'

In [105]:
# Data cleaning. Taking promice data and adding datetime columns + station metadata
#Leaving old code in case it's usefull later

# Process data
# echo -ne 'Processing dairly data...\r'
# python scripts/process_data_daily.py
# echo -ne 'Processing hourly data...\r'
# python scripts/process_data_hourly.py

# Delete unprocessed data
# rm -r data_daily data_hourly

# %% process data hourly
# Reading station metadata
station = pd.read_csv('../data/new_promice/AWS_station_locations.csv')

station.rename(columns={'timestamp':'station_location_timestamp'})

#Reading parquet dataframe
df_hourly = df.copy()

# Delete irrelevant columns from dataframe (i.e. null columns and flag columns)
# null_columns = df_hourly.columns[df_hourly.isnull().all()]
# flag_columns = df_hourly.filter(regex="flag$").columns
# print(null_columns)
# print(flag_columns)

#df_hourly = df_hourly.drop(
#    columns=[
#           'gps_geounit', 
#           'batt_v_ini', 
#           'freq_vw', 
#           'DepthPressureTransducer(m)',
#           'DepthPressureTransducer_Cor(m)', 
#           'IceTemperature7(C)',
#           'IceTemperature8(C)', 
#           'LatitudeGPS(degN)', 
#           'LongitudeGPS(degW)',
#           'ElevationGPS(m)', 
#           'TimeGPS(hhmmssUTC)', 
#           'HorDilOfPrecGPS',
#           'DepthPressureTransducer_Cor_adj(m)', 
#           'IceTemperature10m(C)', 
#           'z_surf_1_adj_flag', 
#           'z_surf_2_adj_flag', 
#           'z_pt_cor_adj_flag'
#    ]
#)
# Add year column to dataframe
df_hourly["time"] = pd.to_datetime(df_hourly.time)

df_hourly["year"] = df_hourly.time.dt.strftime("%Y")

# Add month column to dataframe
df_hourly["month"] = df_hourly.time.dt.strftime("%B")

# Add season column to dataframe
seasons = {
    1: "Winter",
    2: "Winter",
    3: "Spring",
    4: "Spring",
    5: "Spring",
    6: "Summer",
    7: "Summer",
    8: "Summer",
    9: "Autumn",
    10: "Autumn",
    11: "Autumn",
    12: "Winter",
}

# Extract the month from the index and use the dictionary to map it to the corresponding season
df_hourly["season"] = df_hourly.time.dt.month.map(seasons)


#Rename Index Column to Datetime
df_hourly = df_hourly.reset_index(inplace=False)
df_hourly = df_hourly.rename(columns={'time': 'Datetime'}, inplace=False) 

# Rename 'station_name' to file
df_hourly = df_hourly.rename(columns={'station_name': 'stid'}, inplace=False)

# Add Day of Year & Day of Century 
df_hourly['DayOfYear'] = df_hourly['Datetime'].dt.dayofyear 
df_hourly['DayOfCentury'] = df_hourly['Datetime'].dt.dayofyear+365*(df_hourly['Datetime'].dt.year-1)

# # Add day column to dataframe
df_hourly["day"] = df_hourly['Datetime'].dt.strftime("%d")

# # Add hour column to dataframe
df_hourly["hour"] = df_hourly['Datetime'].dt.strftime("%h")

#merging with station metadata

#df_hourly.merge(station, how='left',on=['stid','stid'])

display(df_hourly.head(10))
# Convert the DataFrame to a compressed Parquet file
#output_file = "..\\Data\\new_promice\\all_promice_data.parquet.gzip"
#df.to_parquet(output_file, compression='gzip', engine='pyarrow')


,index,stid,Datetime,p_u,t_u,rh_u,rh_u_cor,qh_u,wspd_u,wdir_u,...,depth_7,depth_8,IceTemperature10m(C),year,month,season,DayOfYear,DayOfCentury,day,hour
0,0,CEN1,2017-05-23 10:00:00+00:00,NaN,NaN,NaN,NaN,NaN,17.33,172.5,...,NaN,NaN,NaN,2017,May,Spring,143,735983,23,May
1,1,CEN1,2017-05-23 11:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2017,May,Spring,143,735983,23,May
2,2,CEN1,2017-05-23 12:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2017,May,Spring,143,735983,23,May
3,3,CEN1,2017-05-23 13:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2017,May,Spring,143,735983,23,May
4,4,CEN1,2017-05-23 14:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2017,May,Spring,143,735983,23,May
5,5,CEN1,2017-05-23 15:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2017,May,Spring,143,735983,23,May
6,6,CEN1,2017-05-23 16:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2017,May,Spring,143,735983,23,May
7,7,CEN1,2017-05-23 17:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2017,May,Spring,143,735983,23,May
8,8,CEN1,2017-05-23 18:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2017,May,Spring,143,735983,23,May
9,9,CEN1,2017-05-23 19:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2017,May,Spring,143,735983,23,May


In [107]:
null_columns = df_hourly.columns[df_hourly.isnull().all()]
flag_columns = df_hourly.filter(regex="flag$").columns
print("null columns:\n", null_columns)
print("flag_columns:\n", flag_columns)

null columns:
 Index(['gps_geounit', 'batt_v_ini', 'freq_vw', 'DepthPressureTransducer(m)',
       'DepthPressureTransducer_Cor(m)', 'IceTemperature7(C)',
       'IceTemperature8(C)', 'LatitudeGPS(degN)', 'LongitudeGPS(degW)',
       'ElevationGPS(m)', 'TimeGPS(hhmmssUTC)', 'HorDilOfPrecGPS',
       'DepthPressureTransducer_Cor_adj(m)', 'IceTemperature10m(C)'],
      dtype='object')
flag_columns:
 Index(['z_surf_1_adj_flag', 'z_surf_2_adj_flag', 'z_pt_cor_adj_flag'], dtype='object')
